In [22]:
%matplotlib inline
import utils; reload(utils)
from utils import *

In [23]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [24]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [25]:
path = "data/dogscats/sample/"
# path = "data/dogscats/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [26]:
batch_size = 4

In [27]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [28]:
batches = get_batches(path+'train',shuffle=False,batch_size=1)
val_batches = get_batches(path+'valid',shuffle=False,batch_size=1)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [29]:
import bcolz
def save_array(fileName,arr): array=bcolz.carray(arr,rootdir=fileName,mode='w'); array.flush()
def load_array(fileName): return bcolz.open(fileName)

In [30]:
trn_data = get_data(path+'train')
val_data = get_data(path+'valid')

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [31]:
trn_data.shape

(16, 3, 224, 224)

In [32]:
save_array(model_path+'train_data.bc', trn_data)
save_array(model_path+'valid_data.bc', val_data)

In [33]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'train_data.bc')

In [34]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [35]:
classes = batches.classes
val_classes = val_batches.classes
labels = onehot(classes)
val_labels = onehot(val_classes)

In [36]:
labels.shape
val_labels.shape

(8, 2)

In [37]:
#predict takes in images as an array
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

In [38]:
trn_features.shape

(16, 1000)

In [39]:
save_array(model_path+'train_lastlayer_features.bc',trn_features)
save_array(model_path+'validation_lastlayer_features.bc',val_features)

In [40]:
trn_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'validation_lastlayer_features.bc')

In [44]:
#1000 inputs b/c vgg saved, 2 outputs
lm = Sequential([ Dense(2, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
batch_size = 4

In [47]:
lm.fit(trn_features, labels, nb_epoch=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

ValueError: Input arrays should have the same number of samples as target arrays. Found 16 input samples and 8 target samples.

NOTE: ask prof Tsai wtf is going on in the lm.summary() layer(type) in lesson2.ipynb